In [2]:
import pandas as pd
import folium
from folium.plugins import HeatMap
###-STEP 1 DOWNLOAD DATA
 
# See details of API at:- https://aqicn.org/api/
base_url = "https://api.waqi.info"
 
# Get token from:- https://aqicn.org/data-platform/token/#/
tok = "13667bf1b664fcXXcf3fd8043f745c38f309fb3f"#sample token , Generate new
 
# (lat, long)-> bottom left, (lat, lon)-> top right
 
# India is 8N 61E to 37N, 97E approx
latlngbox = "8.0000,61.0000,37.0000,97.0000" # For India
trail_url=f"/map/bounds/?latlng={latlngbox}&token={tok}" #
my_data = pd.read_json(base_url + trail_url) # Join parts of URL
print('columns->', my_data.columns) #2 cols ‘status’ and ‘data’


columns-> Index(['status', 'data'], dtype='object')


In [3]:
###-STEP 2:- Create table like DataFrame
all_rows = []
for each_row in my_data['data']:
    all_rows.append([each_row['station']['name'],each_row['lat'],each_row['lon'],each_row['aqi']])
    df = pd.DataFrame(all_rows,
    columns=['station_name', 'lat', 'lon', 'aqi'])


In [4]:
###-STEP 3:- Clean the DataFrame
df['aqi'] = pd.to_numeric(df.aqi,
errors='coerce') # Invalid parse to NaN
print('with NaN->', df.shape) # Comes out as (152, 4)
 
# Remove NaN (Not a Number) entries in col
df1 = df.dropna(subset = ['aqi'])
print('without NaN->', df1.shape) # (144, 4)


with NaN-> (178, 4)
without NaN-> (171, 4)


In [5]:
###-STEP 4:- Make folium heat map
df2 = df1[['lat', 'lon', 'aqi']]
# print(df2.head) # Uncomment to see DataFrame
init_loc = [23, 77] # Approx over Bhopal
max_aqi = int(df1['aqi'].max())
print('max_aqi->', max_aqi)
m = folium.Map(location = init_loc, zoom_start = 5)
 
heat_aqi = HeatMap(df2, min_opacity = 0.1, max_val = max_aqi,
radius = 20, blur = 20, max_zoom = 2)
m.add_child(heat_aqi)
m # Show the map


max_aqi-> 180


/tmp/ipykernel_22937/255148623.py:9: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  heat_aqi = HeatMap(df2, min_opacity = 0.1, max_val = max_aqi,


In [6]:
###-STEP 5 : Plot stations on map
centre_point = [23.25, 77.41] # Approx over Bhopal
m2 = folium.Map(location = centre_point,
tiles = 'Stamen Terrain',
zoom_start= 6)
for idx, row in df1.iterrows():
    lat = row['lat']
    lon = row['lon']
    station = row['station_name'] + ' AQI=' + str(row['aqi'])
    station_aqi = row['aqi']
    if station_aqi > 300: ## Red for very bad AQI
        pop_color = 'red'
    elif station_aqi > 200:
        pop_color = 'orange' ## Orange for moderate AQI
    else:
        pop_color = 'green' ## Green for good AQI
    folium.Marker(location= [lat, lon],
    popup = station,
    icon = folium.Icon(color = pop_color)).add_to(m2)
m2 # Display map

In [9]:
! pipreqs . --force

INFO: Successfully saved requirements file in ./requirements.txt
